In [4]:
import time
import csv

# time.clock deprecated, use time.perf_counter or time.process_time

f = open("chopsticks.csv", "r")
list_from_file = list(csv.reader(f))
list_from_RAM = "1,2,3,4,5,6,7,8,9,10".split(",")

before_file_read = time.process_time()
f = open("chopsticks.csv", "r")
list_from_file = list(csv.reader(f))
after_file_read = time.process_time()
file_time = after_file_read - before_file_read
print(file_time)

before_string_parse = time.process_time()
list_from_string = "1,2,3,4,5,6,7,8,9,10".split(",")
after_string_parse = time.process_time()
RAM_time = after_string_parse - before_string_parse
print(RAM_time)

bfr = time.perf_counter()
f = open("chopsticks.csv", "r")
list_from_file = list(csv.reader(f))
afr = time.perf_counter()
file_time = afr - bfr
print("file time perf_counter:", file_time)

0.0009640000000001869
0.00022199999999994446
file time perf_counter: 0.0006500860000073772


In [5]:
class Counter():
    def __init__(self):
        self.count = 0
    def increment(self):
        self.count += 1
    def get_count(self):
        return self.count

def count_up_100000(counter):
    for i in range(100000):
        counter.increment()
        
counter = Counter()
initial_count = counter.get_count()
count_up_100000(counter)
final_count = counter.get_count()

In [6]:
# Multithreading multiple processes

import threading

counter = Counter()
count_thread = threading.Thread(target=count_up_100000, args=[counter])
count_thread.start()
count_thread.join()
after_join = counter.get_count()
print(after_join)

100000


In [7]:
def conduct_trial():
    counter = Counter()
    count_thread = threading.Thread(target=count_up_100000, args=[counter])
    count_thread.start()
    intermediate_value = counter.get_count()
    count_thread.join()
    return intermediate_value

trial1 = conduct_trial()
print(trial1)
trial2 = conduct_trial()
print(trial2)
trial3 = conduct_trial()
print(trial3)

12726
15418
47829


In [ ]:
# Using locks to enforce determinism
def count_up_100000(counter, lock):
    for i in range(10000):
        lock.acquire()
        for i in range(10):
            counter.increment()
        lock.release()

def conduct_trial():
    counter = Counter()
    lock = threading.Lock()
    count_thread = threading.Thread(target=count_up_100000, args=[counter, lock])
    count_thread.start()
    lock.acquire()
    intermediate_value = counter.get_count()
    lock.release()
    count_thread.join()
    return intermediate_value

trial1 = conduct_trial()
print(trial1)
trial2 = conduct_trial()
print(trial2)
trial3 = conduct_trial()
print(trial3)

In [ ]:
def count_up_100000(counter):
    for i in range(100000):
        counter.increment()

def conduct_trial():
    counter = Counter()
    count_thread1 = threading.Thread(target=count_up_100000, args=[counter])
    count_thread2 = threading.Thread(target=count_up_100000, args=[counter])

    count_thread1.start()
    count_thread2.start()

    # Join the threads here
    count_thread1.join()
    count_thread2.join()
    
    final_count = counter.get_count()
    return final_count

trial1 = conduct_trial()
print(trial1)
trial2 = conduct_trial()
print(trial2)
trial3 = conduct_trial()
print(trial3)

In [ ]:
# imitate atomicity with locks

class Counter():
    def __init__(self):
        self.count = 0
        self.lock = threading.Lock()
    def increment(self):
        self.lock.acquire()
        old_count = self.count
        self.count = old_count + 1
        self.lock.release()
    def get_count(self):
        return self.count

def count_up_100000(counter):
    for i in range(100000):
        counter.increment()

def conduct_trial():
    counter = Counter()
    count_thread1 = threading.Thread(target=count_up_100000, args=[counter])
    count_thread2 = threading.Thread(target=count_up_100000, args=[counter])

    count_thread1.start()
    count_thread2.start()

    count_thread1.join()
    count_thread2.join()

    final_count = counter.get_count()
    return final_count

trial1 = conduct_trial()
print(trial1)
trial2 = conduct_trial()
print(trial2)
trial3 = conduct_trial()
print(trial3)